# Création du serveur GitLab

### Pour sauvegarder les données du server gitlab et garder la persistance des données malgré sont arrêt nous pouvons faire la commande :

```bash
sudo mkdir -p /srv/gitlab/config /srv/gitlab/logs /srv/gitlab/data
```

### Installation de l’image GitLab avec le paramétrage du server :

```bahs
sudo docker run --detach \
--hostname 0xhadock.gitlab.com \
--publish 443:443 --publish 8088:80 --publish 2022:22 \
--name gitlab \
--restart always \
--volume /srv/gitlab/config:/etc/gitlab \
--volume /srv/gitlab/logs:/var/log/gitlab \
--volume /srv/gitlab/data:/var/opt/gitlab \
--network host \
gitlab/gitlab-ee:latest
```

### Configurer GitLab afin qu'il utilise le port ```2022``` et le port ```8088```

```bash
sudo nano /srv/gitlab/config/gitlab.rb
```

Recherche de la ligne /gitlab_rails avec ```Ctr+w``` et modification


```bash
gitlab_rails['gitlab_shell_ssh_port'] = 2022
external_url "http://0xhadock.gitlab.com:8088"
```

### Appliquez les changements :

```bash
sudo docker exec -it gitlab gitlab-ctl reconfigure
```

### Vérifier l’adresse ip du server :

```bash
sudo docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' gitlab
```

#### Tester la connexion de cette manière :

```bash
curl -I http://<ip>:8088
```

#### Sur le mac mettre la redirection de service :

```bash
sudo vim /etc/hosts 
```

```192.168.0.20  0xhadock.gitlab.com```

### Chercher le mot de passe par défaut dans :

```bash
sudo cat /srv/gitlab/config/initial_root_password
```

#### Connexion au server gitlab :

```id : root```  
```password : initial_root_password```

# Création du Runner GitLab

Créer un nouveau runner dans CI/CD, puis sélectionner container Docker. 

### Création du runner

```bash
sudo docker run -d --name gitlab-runner \
  --restart always \
  --add-host 0xhadock.gitlab.com:192.168.0.20 \
  -v /srv/gitlab-runner/config:/etc/gitlab-runner \
  -v /var/run/docker.sock:/var/run/docker.sock \
  gitlab/gitlab-runner:latest
```

puis entrer la commande suivante :

```bash
sudo docker exec -it gitlab-runner gitlab-runner register
```

```entrer l’adresse de l’instance gitlab : http://0xhadock.gitlab.com:8088```  
```entrer le token : glrt-t1_igWASHu3WhpPkrfxs58b```  
```entrer un executor : docker```  
```entrer l’image docker par défaut : debian:latest```

### Entrer dans le conteneur docker du runner

```bash
sudo docker exec -it gitlab-runner /bin/bash
```

### Installer vim

```bash
apt-get update && apt-get install -y vim
```

### Modifier les paramétres :

```bash
vim /etc/gitlab-runner/config.toml
```

```toml
[[runners]]
name = "runner_00"
url = "http://0xhadock.gitlab.com:8088"
id = 2
token = "glrt-t1_igWASHu3WhpPkrfxs58b"
token_obtained_at = 2024-10-29T15:34:50Z
token_expires_at = 0001-01-01T00:00:00Z
executor = "docker"
[runners.custom_build_dir]
[runners.cache]
MaxUploadedArchiveSize = 0
[runners.cache.s3]
[runners.cache.gcs]
[runners.cache.azure]
[runners.docker]
tls_verify = false
image = "docker:latest"  # Changer à une image contenant Docker
privileged = true          # Activer le mode privilégié
disable_entrypoint_overwrite = false
oom_kill_disable = false
disable_cache = false
extra_hosts = ["0xhadock.gitlab.com:192.168.0.20"]
dns_search = ["0xhadock.gitlab.com"]
volumes = ["/cache", "/var/run/docker.sock:/var/run/docker.sock"]  # Monter le socket Docker
shm_size = 0
network_mtu = 0
```

### Restart du conteneur 

```bash
sudo docker restart gitlab-runner
```

### Test du runner 

```yaml
stages:
  - test

test-job:
  stage: test
  tags:
      - runner_00
  script:
    - echo "Start .."
    - sleep 60
    - echo "End!"
```